<a href="https://colab.research.google.com/github/maximlambov/MARK/blob/master/TensorflowJS_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run cell (commands) below and then hit F5 (refresh the page) right after that

In [ ]:
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

SyntaxError: ignored

That's it! :) Now in the next cells you can write JavaScript

# Example of usage

**Remember not to use `const` or `let`! Use `var` instead**

This is how you can execute shell commands:

In [ ]:
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
sh('npm init -y');

In [5]:
sh('node -v; npm -v');

v14.16.0

6.14.8



In [125]:
sh('npm install @tensorflow/tfjs-node-gpu');
sh('npm install node-fetch@2');

npm
 
WARN
 content@1.0.0 No description

npm
 WARN content@1.0.0 No repository field.





+ node-fetch@2.6.7
updated 1 package and audited 119 packages in 1.566s


9 packages are looking for funding
  run `npm fund` for details


found 0 vulnerabilities




In [127]:
var tf = require('@tensorflow/tfjs-node-gpu');
tf.backend().isUsingGpuDevice;
tf.enableProdMode();



In [ ]:
var fetch = require('node-fetch@2');

var EPOCHS = 3_000;
var ticker = 'AUDUSD=X';

$$.async();
(async () => {
  try {
    $$.async();
    var response = await fetch({
      url: "https://query1.finance.yahoo.com/v8/finance/chart/AUDUSD=X?interval=1d&range=10y",
      method: 'GET',
    });

    var data = await response.json();
    $$.done();

    console.log(data)

  } catch (e) {
    console.log("e",e)
   }

})()
$$.done();

// --------------------------------

// var { minToMax, maxToMin } = normalization({
//   Xmin: Math.min(...adjustedClose.flat()),
//   Xmax: Math.max(...adjustedClose.flat()),
//   d1: 0,
//   d2: 1,
// });

In [ ]:
  var model = tf.sequential();

  model.add(
    tf.layers.dense({
      name: 'InputLayer',
      units: 300,
      inputShape: [20],
      activation: 'sigmoid',
    })
  );

  model.add(
    tf.layers.dense({
      name: 'HiddenLayer2',
      units: 300,
      activation: 'sigmoid',
    })
  );

  model.add(
    tf.layers.dense({ name: 'OutputLayer', units: 5, activation: 'sigmoid' })
  );

  model.summary();

  model.compile({
    loss: 'meanSquaredError',
    optimizer: tf.train.sgd(0.3),
  });